# 0. Install and Import Dependencies

In [1]:
!pip install mediapipe opencv-python pandas scikit-learn

In [2]:
import mediapipe as mp # Import mediapipe
import cv2 # Import opencv
import numpy as np
import csv
import os
import pandas as pd
import pickle


In [3]:
mp_drawing = mp.solutions.drawing_utils # Drawing helpers
mp_holistic = mp.solutions.holistic # Mediapipe Solutions
mp_pose = mp.solutions.pose

# 4. Make Detections with Model

In [4]:
with open('body_language.pkl', 'rb') as f:
    model = pickle.load(f)

In [5]:
model

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('randomforestclassifier', RandomForestClassifier())])

In [6]:
#팔, 어깨 각도 계산
def calculate_angle(a,b,c):
    a = np.array(a) # First
    b = np.array(b) # Mid
    c = np.array(c) # End
    
    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians*180.0/np.pi)
    
    if angle >180.0:
        angle = 360-angle
        
    return angle 

In [16]:
cap = cv2.VideoCapture('pushup_pro.mp4')
# Initiate holistic model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    while cap.isOpened():
        ret, frame = cap.read()
        
        # Recolor Feed
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False        
        
        # Make Detections
        results = holistic.process(image)
        # print(results.face_landmarks)
        
        # face_landmarks, pose_landmarks, left_hand_landmarks, right_hand_landmarks
        
        # Recolor image back to BGR for rendering
        image.flags.writeable = True   
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # 1. Right hand
        mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                                 )

        # 2. Left Hand
        mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                                 )

        # 3. Pose Detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                                 )
        # Export coordinates
        try:
            # Extract Pose landmarks
            pose = results.pose_landmarks.landmark
            pose_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in pose]).flatten())
            
            # Concatenate rows
            row = pose_row
            
            
#             # Append class name 
#             row.insert(0, class_name)
            
#             # Export to CSV
#             with open('coords.csv', mode='a', newline='') as f:
#                 csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
#                 csv_writer.writerow(row) 
            
            
            # Make Detections
            X = pd.DataFrame([row])
            body_language_class = model.predict(X)[0]
            body_language_prob = model.predict_proba(X)[0]
            print(body_language_class, body_language_prob)
            
            # Grab ear coords
            coords = tuple(np.multiply(
                            np.array(
                                (results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EAR].x, 
                                 results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EAR].y))
                        , [640,480]).astype(int))
            
            cv2.rectangle(image, 
                          (coords[0], coords[1]+5), 
                          (coords[0]+len(body_language_class)*20, coords[1]-30), 
                          (245, 117, 16), -1)
            cv2.putText(image, body_language_class, coords, 
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
            
            #Get status box
            cv2.rectangle(image, (0,0), (250, 60), (245, 117, 16), -1)
            
            #Display Class
            cv2.putText(image, 'CLASS'
                        , (95,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
            cv2.putText(image, body_language_class.split(' ')[0]
                        , (90,40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
            
            #Display Probability
            cv2.putText(image, 'PROB'
                       , (15,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
            cv2.putText(image, str(round(body_language_prob[np.argmax(body_language_prob)],2))
                       , (10,40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
            
            # Get coordinates
            shoulder = [pose[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,pose[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
            elbow = [pose[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,pose[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
            wrist = [pose[mp_pose.PoseLandmark.LEFT_WRIST.value].x,pose[mp_pose.PoseLandmark.LEFT_WRIST.value].y]
            
            # Calculate angle
            angle = calculate_angle(shoulder, elbow, wrist)
            
            # Visualize angle
            cv2.putText(image, str(angle), 
                           tuple(np.multiply(elbow, [640, 480]).astype(int)), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA
                                )
            
            # Curl counter logic
            if angle > 160:
                stage = "down"
            if angle < 30 and stage =='down':
                stage="up"
                counter +=1
                print(counter)
               
        
        except:
            pass
        #횟수 세는 알고리즘(디스플레이 안되는 부분)
            # Render curl counter
            # Setup status box
            cv2.rectangle(image, (300,70), (550,130), (245,117,16), -1)

                # Rep data
            cv2.putText(image, 'REPS', (315,300), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
            cv2.putText(image, str(counter), 
                        (300,338), 
                        cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255), 2, cv2.LINE_AA)

            # Stage data
            cv2.putText(image, 'STAGE', (250,300), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
            cv2.putText(image, stage, 
                        (255,340), 
                        cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255), 2, cv2.LINE_AA)


            # Render detections
            mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                    mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2), 
                                    mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2) 
                                     )             
            
        
        cv2.imshow('Raw Webcam Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

nice pushup [0.27 0.29 0.44]
nice pushup [0.27 0.29 0.44]
nice pushup [0.27 0.29 0.44]
nice pushup [0.27 0.28 0.45]
nice pushup [0.27 0.27 0.46]
nice pushup [0.27 0.28 0.45]
nice pushup [0.27 0.28 0.45]
nice pushup [0.28 0.27 0.45]
nice pushup [0.29 0.27 0.44]
nice pushup [0.29 0.27 0.44]
nice pushup [0.29 0.26 0.45]
nice pushup [0.29 0.26 0.45]
nice pushup [0.29 0.26 0.45]
nice pushup [0.29 0.26 0.45]
nice pushup [0.29 0.26 0.45]
nice pushup [0.27 0.28 0.45]
nice pushup [0.27 0.28 0.45]
nice pushup [0.27 0.28 0.45]
nice pushup [0.27 0.28 0.45]
nice pushup [0.28 0.28 0.44]
nice pushup [0.28 0.28 0.44]
nice pushup [0.28 0.28 0.44]
nice pushup [0.28 0.28 0.44]
nice pushup [0.28 0.28 0.44]
nice pushup [0.28 0.28 0.44]
nice pushup [0.28 0.28 0.44]
nice pushup [0.28 0.28 0.44]
nice pushup [0.28 0.28 0.44]
nice pushup [0.27 0.28 0.45]
nice pushup [0.27 0.28 0.45]
nice pushup [0.27 0.27 0.46]
nice pushup [0.3  0.26 0.44]
nice pushup [0.32 0.26 0.42]
nice pushup [0.31 0.27 0.42]
nice pushup [0

nice benchpress [0.39 0.24 0.37]
nice benchpress [0.39 0.23 0.38]
nice benchpress [0.39 0.22 0.39]
nice benchpress [0.38 0.24 0.38]
nice benchpress [0.38 0.24 0.38]
nice pushup [0.36 0.25 0.39]
nice pushup [0.37 0.25 0.38]
nice benchpress [0.39 0.24 0.37]
nice benchpress [0.39 0.24 0.37]
nice benchpress [0.39 0.25 0.36]
nice benchpress [0.39 0.26 0.35]
nice benchpress [0.38 0.27 0.35]
nice benchpress [0.38 0.27 0.35]
nice benchpress [0.38 0.27 0.35]
nice pushup [0.34 0.28 0.38]
nice pushup [0.28 0.31 0.41]
nice pushup [0.28 0.33 0.39]
nice pushup [0.29 0.3  0.41]
nice pushup [0.29 0.3  0.41]
nice pushup [0.3 0.3 0.4]
nice pushup [0.3  0.29 0.41]
nice pushup [0.3  0.29 0.41]
nice pushup [0.3  0.29 0.41]
nice pushup [0.3  0.29 0.41]
nice pushup [0.3  0.28 0.42]
nice pushup [0.3  0.28 0.42]
nice pushup [0.3  0.28 0.42]
nice pushup [0.3  0.28 0.42]
nice pushup [0.3  0.28 0.42]
nice pushup [0.31 0.27 0.42]
nice pushup [0.33 0.26 0.41]
nice pushup [0.32 0.26 0.42]
nice pushup [0.34 0.23 0.43

nice pushup [0.24 0.32 0.44]
nice pushup [0.25 0.31 0.44]
nice pushup [0.25 0.31 0.44]
nice pushup [0.25 0.31 0.44]
nice pushup [0.25 0.31 0.44]
nice pushup [0.33 0.31 0.36]
nice benchpress [0.37 0.27 0.36]
nice pushup [0.37 0.25 0.38]
nice benchpress [0.38 0.25 0.37]
nice pushup [0.38 0.23 0.39]
nice benchpress [0.39 0.23 0.38]
nice benchpress [0.38 0.24 0.38]
nice pushup [0.35 0.25 0.4 ]
nice pushup [0.35 0.25 0.4 ]
nice pushup [0.36 0.24 0.4 ]
nice pushup [0.37 0.23 0.4 ]
nice pushup [0.37 0.23 0.4 ]
nice benchpress [0.39 0.22 0.39]
nice pushup [0.39 0.2  0.41]
nice pushup [0.38 0.2  0.42]
nice pushup [0.32 0.22 0.46]
nice pushup [0.32 0.22 0.46]
nice pushup [0.36 0.2  0.44]
nice pushup [0.36 0.21 0.43]
nice pushup [0.37 0.2  0.43]
nice pushup [0.37 0.2  0.43]
nice pushup [0.38 0.2  0.42]
nice pushup [0.38 0.21 0.41]
nice pushup [0.38 0.21 0.41]
nice pushup [0.38 0.21 0.41]
nice benchpress [0.4 0.2 0.4]
nice benchpress [0.41 0.19 0.4 ]
nice benchpress [0.42 0.18 0.4 ]
nice benchpres

nice pushup [0.23 0.32 0.45]
nice pushup [0.23 0.32 0.45]
nice pushup [0.23 0.32 0.45]
nice pushup [0.24 0.32 0.44]
nice pushup [0.24 0.32 0.44]
nice pushup [0.24 0.32 0.44]
nice pushup [0.25 0.31 0.44]
nice pushup [0.25 0.31 0.44]
nice pushup [0.25 0.31 0.44]
nice benchpress [0.36 0.29 0.35]
nice benchpress [0.37 0.27 0.36]
nice benchpress [0.38 0.25 0.37]
nice benchpress [0.39 0.23 0.38]
nice benchpress [0.4  0.21 0.39]
nice benchpress [0.4  0.22 0.38]
nice benchpress [0.4  0.23 0.37]
nice pushup [0.37 0.23 0.4 ]
nice benchpress [0.4  0.21 0.39]
nice benchpress [0.42 0.19 0.39]
nice benchpress [0.41 0.19 0.4 ]
nice benchpress [0.44 0.17 0.39]
nice benchpress [0.44 0.17 0.39]
nice benchpress [0.44 0.17 0.39]
nice pushup [0.41 0.17 0.42]
nice pushup [0.35 0.19 0.46]
nice pushup [0.34 0.2  0.46]
nice pushup [0.34 0.2  0.46]
nice pushup [0.34 0.19 0.47]
nice pushup [0.35 0.2  0.45]
nice pushup [0.37 0.19 0.44]
nice pushup [0.39 0.18 0.43]
nice pushup [0.39 0.18 0.43]
nice pushup [0.39 0.

nice pushup [0.35 0.28 0.37]
nice pushup [0.35 0.28 0.37]
nice pushup [0.35 0.28 0.37]
nice pushup [0.35 0.28 0.37]
nice pushup [0.35 0.28 0.37]
nice pushup [0.35 0.28 0.37]
nice benchpress [0.36 0.28 0.36]
nice benchpress [0.36 0.28 0.36]
nice benchpress [0.36 0.28 0.36]
nice benchpress [0.36 0.28 0.36]
nice benchpress [0.36 0.28 0.36]
nice pushup [0.35 0.29 0.36]
nice pushup [0.35 0.29 0.36]
nice pushup [0.35 0.29 0.36]
nice pushup [0.35 0.29 0.36]
nice pushup [0.35 0.29 0.36]
nice pushup [0.34 0.3  0.36]
nice pushup [0.31 0.31 0.38]
nice pushup [0.3  0.31 0.39]
nice pushup [0.27 0.31 0.42]
nice pushup [0.27 0.31 0.42]
nice pushup [0.27 0.31 0.42]
nice pushup [0.26 0.32 0.42]
nice pushup [0.26 0.32 0.42]
nice pushup [0.26 0.32 0.42]
nice pushup [0.26 0.32 0.42]
nice pushup [0.26 0.32 0.42]
nice pushup [0.26 0.32 0.42]
nice pushup [0.26 0.32 0.42]
nice pushup [0.26 0.32 0.42]
nice pushup [0.26 0.32 0.42]
nice pushup [0.26 0.32 0.42]
nice pushup [0.27 0.31 0.42]
nice pushup [0.27 0.31 

...

nice benchpress [0.49 0.15 0.36]
nice benchpress [0.48 0.15 0.37]
nice benchpress [0.45 0.15 0.4 ]
nice benchpress [0.45 0.15 0.4 ]
nice benchpress [0.45 0.16 0.39]
nice benchpress [0.45 0.16 0.39]
nice pushup [0.36 0.18 0.46]
nice pushup [0.34 0.19 0.47]
nice pushup [0.32 0.2  0.48]
nice pushup [0.31 0.21 0.48]
nice pushup [0.36 0.2  0.44]
nice pushup [0.36 0.22 0.42]
nice pushup [0.35 0.22 0.43]
nice pushup [0.34 0.23 0.43]
nice pushup [0.33 0.24 0.43]
nice pushup [0.32 0.22 0.46]
nice pushup [0.3  0.22 0.48]
nice pushup [0.3  0.21 0.49]
nice pushup [0.29 0.23 0.48]
nice pushup [0.28 0.25 0.47]
nice pushup [0.25 0.27 0.48]
nice pushup [0.27 0.27 0.46]
nice pushup [0.27 0.27 0.46]
nice pushup [0.32 0.26 0.42]
nice pushup [0.35 0.25 0.4 ]
nice pushup [0.35 0.26 0.39]
nice pushup [0.35 0.26 0.39]
nice pushup [0.35 0.27 0.38]
nice pushup [0.35 0.27 0.38]
nice pushup [0.35 0.27 0.38]
nice pushup [0.35 0.27 0.38]
nice pushup [0.35 0.28 0.37]
nice pushup [0.35 0.28 0.37]
nice pushup [0.35 0

nice benchpress [0.42 0.18 0.4 ]
nice benchpress [0.42 0.18 0.4 ]
nice benchpress [0.42 0.18 0.4 ]
nice benchpress [0.42 0.18 0.4 ]
nice benchpress [0.42 0.18 0.4 ]
nice benchpress [0.43 0.18 0.39]
nice benchpress [0.43 0.18 0.39]
nice benchpress [0.44 0.17 0.39]
nice benchpress [0.45 0.17 0.38]
nice benchpress [0.45 0.18 0.37]
nice benchpress [0.46 0.18 0.36]
nice benchpress [0.46 0.18 0.36]
nice benchpress [0.46 0.17 0.37]
nice benchpress [0.45 0.17 0.38]
nice benchpress [0.43 0.17 0.4 ]
nice pushup [0.41 0.17 0.42]
nice pushup [0.41 0.17 0.42]
nice pushup [0.41 0.17 0.42]
nice benchpress [0.41 0.18 0.41]
nice benchpress [0.41 0.18 0.41]
nice pushup [0.39 0.19 0.42]
nice pushup [0.3  0.21 0.49]
nice pushup [0.32 0.22 0.46]
nice pushup [0.31 0.23 0.46]
nice pushup [0.34 0.22 0.44]
nice pushup [0.33 0.23 0.44]
nice pushup [0.3  0.22 0.48]
nice pushup [0.3  0.24 0.46]
nice pushup [0.28 0.25 0.47]
nice pushup [0.29 0.22 0.49]
nice pushup [0.26 0.26 0.48]
nice pushup [0.25 0.26 0.49]
nice

nice benchpress [0.55 0.13 0.32]
nice benchpress [0.54 0.14 0.32]
nice benchpress [0.54 0.14 0.32]
nice benchpress [0.51 0.14 0.35]
nice benchpress [0.51 0.14 0.35]
nice benchpress [0.46 0.15 0.39]
nice benchpress [0.45 0.16 0.39]
nice benchpress [0.45 0.16 0.39]
nice benchpress [0.45 0.16 0.39]
nice benchpress [0.45 0.16 0.39]
nice benchpress [0.45 0.16 0.39]
nice benchpress [0.46 0.17 0.37]
nice benchpress [0.47 0.17 0.36]
nice benchpress [0.47 0.17 0.36]
nice benchpress [0.47 0.17 0.36]
nice benchpress [0.47 0.17 0.36]
nice benchpress [0.48 0.15 0.37]
nice benchpress [0.5  0.14 0.36]
nice benchpress [0.49 0.15 0.36]
nice benchpress [0.49 0.15 0.36]
nice benchpress [0.49 0.15 0.36]
nice benchpress [0.49 0.15 0.36]
nice benchpress [0.47 0.15 0.38]
nice benchpress [0.44 0.15 0.41]
nice benchpress [0.44 0.15 0.41]
nice benchpress [0.44 0.16 0.4 ]
nice benchpress [0.45 0.16 0.39]
nice benchpress [0.44 0.16 0.4 ]
nice pushup [0.4  0.17 0.43]
nice pushup [0.32 0.2  0.48]
nice pushup [0.31 

nice benchpress [0.45 0.18 0.37]
nice benchpress [0.44 0.18 0.38]
nice benchpress [0.44 0.18 0.38]
nice benchpress [0.44 0.18 0.38]
nice benchpress [0.48 0.16 0.36]
nice benchpress [0.48 0.16 0.36]
nice benchpress [0.48 0.15 0.37]
nice benchpress [0.49 0.14 0.37]
nice benchpress [0.51 0.13 0.36]
nice benchpress [0.52 0.13 0.35]
nice benchpress [0.57 0.12 0.31]
nice benchpress [0.55 0.13 0.32]
nice benchpress [0.54 0.14 0.32]
nice benchpress [0.54 0.14 0.32]
nice benchpress [0.54 0.13 0.33]
nice benchpress [0.54 0.13 0.33]
nice benchpress [0.53 0.13 0.34]
nice benchpress [0.5  0.13 0.37]
nice benchpress [0.5  0.14 0.36]
nice benchpress [0.45 0.16 0.39]
nice benchpress [0.44 0.17 0.39]
nice benchpress [0.44 0.17 0.39]
nice benchpress [0.44 0.17 0.39]
nice benchpress [0.45 0.17 0.38]
nice benchpress [0.45 0.17 0.38]
nice benchpress [0.45 0.17 0.38]
nice benchpress [0.46 0.17 0.37]
nice benchpress [0.46 0.17 0.37]
nice benchpress [0.47 0.16 0.37]
nice benchpress [0.47 0.16 0.37]
nice bench

error: OpenCV(4.5.4-dev) /Users/runner/work/opencv-python/opencv-python/opencv/modules/imgproc/src/color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cvtColor'


In [ ]:
import numpy as np

tuple(np.multiply(np.array((results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EAR].x, 
results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EAR].y)), [640,480]).astype(int))